In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np

# 데이터 로드
train_data = np.load('/content/drive/MyDrive/team_project2/train.npy', allow_pickle=True).item()
test_data = np.load('/content/drive/MyDrive/team_project2/test.npy', allow_pickle=True).item()

X_train = train_data['input']
y_train = train_data['label']
X_test = test_data['input']

# 데이터 형태 변경
X_train = X_train.squeeze(1)  # (4608, 22, 1125)
X_test = X_test.squeeze(1)    # (576, 22, 1125)


In [4]:

print("정수 라벨(0~3)으로 변환된 라벨 shape:", y_train.shape)


정수 라벨(0~3)으로 변환된 라벨 shape: (4608, 4)


In [8]:
import pywt
import numpy as np

def compute_cwt(data, wavelet='morl', scales=np.arange(1, 31)):
    cwt_matrices = []
    for epoch in data:
        cwt_epoch = []
        for channel in epoch:
            coefficients, _ = pywt.cwt(channel, scales, wavelet)
            cwt_epoch.append(coefficients)
        cwt_matrices.append(cwt_epoch)
    return np.array(cwt_matrices)

# CWT를 사용하여 시간-주파수 표현 계산
scales = np.arange(1, 31)
X_train_cwt = compute_cwt(X_train, scales=scales)
X_test_cwt = compute_cwt(X_test, scales=scales)

# CNN 입력 형태로 변환
X_train_cwt = np.transpose(X_train_cwt, (0, 3, 1, 2))  # (samples, channels, scales, time)
X_test_cwt = np.transpose(X_test_cwt, (0, 3, 1, 2))  # (samples, channels, scales, time)

# 채널을 마지막 축으로 이동하여 CNN에 맞게 변환
X_train_cwt = X_train_cwt.reshape((X_train_cwt.shape[0], X_train_cwt.shape))


TypeError: 'tuple' object cannot be interpreted as an integer

In [13]:
X_train_cwt = X_train_cwt.reshape((X_train_cwt.shape[0], X_train_cwt.shape[1], X_train_cwt.shape[2], X_train_cwt.shape[3]))
X_test_cwt = X_test_cwt.reshape((X_test_cwt.shape[0], X_test_cwt.shape[1], X_test_cwt.shape[2], X_test_cwt.shape[3]))

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# 라벨을 원-핫 인코딩


# CNN 모델 정의
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(X_train_cwt.shape[1], X_train_cwt.shape[2], X_train_cwt.shape[3])))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
history = model.fit(X_train_cwt, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/50
58/58 [==============================] - 42s 688ms/step - loss: 1.8674 - accuracy: 0.2716 - val_loss: 2.0452 - val_accuracy: 0.2625
Epoch 2/50
58/58 [==============================] - 36s 616ms/step - loss: 1.5485 - accuracy: 0.3353 - val_loss: 1.6074 - val_accuracy: 0.3145
Epoch 3/50
58/58 [==============================] - 35s 611ms/step - loss: 1.3987 - accuracy: 0.4031 - val_loss: 1.6141 - val_accuracy: 0.3015
Epoch 4/50
58/58 [==============================] - 36s 619ms/step - loss: 1.3174 - accuracy: 0.4409 - val_loss: 1.4563 - val_accuracy: 0.3547
Epoch 5/50
58/58 [==============================] - 36s 628ms/step - loss: 1.2223 - accuracy: 0.4783 - val_loss: 1.5833 - val_accuracy: 0.3091
Epoch 6/50
58/58 [==============================] - 36s 616ms/step - loss: 1.1347 - accuracy: 0.5220 - val_loss: 1.5142 - val_accuracy: 0.2798
Epoch 7/50
58/58 [==============================] - 36s 624ms/step - loss: 1.0232 - accuracy: 0.5676 - val_loss: 1.5607 - val_accuracy: 0.3145

In [17]:
# 테스트 데이터 예측
y_test_pred = model.predict(X_test_cwt)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)

# 예측 결과를 DataFrame으로 변환
import pandas as pd
df = pd.DataFrame(y_test_pred_classes, columns=['Predicted'])

# CSV 파일로 저장
df.to_csv('predictions1.csv', index=True)


18/18 [==============================] - 1s 68ms/step


In [ ]:
df